# Bloch Simulation Analysis

**Mode**: Load data from HDF5 file

**Data file**: `test_notebook_data.h5`

This notebook loads pre-computed simulation data and provides visualization and analysis tools.

## Setup and Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
from pathlib import Path

# Set matplotlib style
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## Load Simulation Data

In [ ]:
# Load data from HDF5 file
data_file = 'test_notebook_data.h5'

if not Path(data_file).exists():
    raise FileNotFoundError(f"Data file not found: {data_file}")

print(f"Loading data from: {data_file}")

data = {}
with h5py.File(data_file, 'r') as f:
    # Load magnetization data
    data['mx'] = f['mx'][...]
    data['my'] = f['my'][...]
    data['mz'] = f['mz'][...]
    data['signal'] = f['signal'][...]

    # Load coordinate arrays
    data['time'] = f['time'][...]
    data['positions'] = f['positions'][...]
    data['frequencies'] = f['frequencies'][...]

    # Load tissue parameters
    data['tissue'] = {}
    for key in ['name', 't1', 't2', 'density', 't2_star']:
        try:
            data['tissue'][key] = f['tissue'].attrs[key]
        except KeyError:
            pass

    print(f"Data loaded successfully!")
    print(f"  Shape: {data['mx'].shape}")
    print(f"  Duration: {data['time'][-1]*1000:.3f} ms")


## Simulation Parameters

In [ ]:
# Display simulation parameters
print("="*60)
print("SIMULATION PARAMETERS")
print("="*60)

print("\nTissue:")
print(f"  Name: {data['tissue'].get('name', 'Unknown')}")
print(f"  T1: {data['tissue'].get('t1', 0)*1000:.1f} ms")
print(f"  T2: {data['tissue'].get('t2', 0)*1000:.1f} ms")

print("\nSequence:")
for key, value in {'sequence_type': 'Spin Echo', 'te': 0.02, 'tr': 0.1, 'flip_angle': 90.0}.items():
    print(f"  {key}: {value}")

print("\nSimulation:")
for key, value in {'mode': 'time-resolved', 'num_positions': 1, 'num_frequencies': 1, 'time_step_us': 1.0}.items():
    print(f"  {key}: {value}")

print("="*60)


## Quick Analysis

In [ ]:
# Quick analysis
print("\nData Statistics:")
print(f"  Time points: {len(data['time'])}")
print(f"  Positions: {data['positions'].shape[0]}")
print(f"  Frequencies: {len(data['frequencies'])}")

if data['mx'].ndim == 3:  # Time-resolved
    mx_final = data['mx'][-1]
    my_final = data['my'][-1]
    mz_final = data['mz'][-1]

    print("\nFinal Magnetization:")
    print(f"  Mx range: [{mx_final.min():.4f}, {mx_final.max():.4f}]")
    print(f"  My range: [{my_final.min():.4f}, {my_final.max():.4f}]")
    print(f"  Mz range: [{mz_final.min():.4f}, {mz_final.max():.4f}]")

    # Find peak transverse magnetization
    mxy = np.sqrt(data['mx']**2 + data['my']**2)
    max_mxy = mxy.max()
    max_idx = np.unravel_index(mxy.argmax(), mxy.shape)

    print(f"\n  Peak |Mxy|: {max_mxy:.4f}")
    print(f"  At time: {data['time'][max_idx[0]]*1000:.3f} ms")


## Magnetization Evolution

In [ ]:
# Plot magnetization evolution
position_idx = 0  # Change to plot different position
freq_idx = 0      # Change to plot different frequency

if data['mx'].ndim == 3:  # Time-resolved
    time_ms = data['time'] * 1000
    mx = data['mx'][:, position_idx, freq_idx]
    my = data['my'][:, position_idx, freq_idx]
    mz = data['mz'][:, position_idx, freq_idx]
    mxy = np.sqrt(mx**2 + my**2)

    fig, axes = plt.subplots(2, 2, figsize=(12, 8))

    axes[0, 0].plot(time_ms, mx, 'b-', linewidth=1.5)
    axes[0, 0].set_xlabel('Time (ms)')
    axes[0, 0].set_ylabel('Mx')
    axes[0, 0].set_title('Transverse Magnetization (x)')
    axes[0, 0].grid(True, alpha=0.3)

    axes[0, 1].plot(time_ms, my, 'r-', linewidth=1.5)
    axes[0, 1].set_xlabel('Time (ms)')
    axes[0, 1].set_ylabel('My')
    axes[0, 1].set_title('Transverse Magnetization (y)')
    axes[0, 1].grid(True, alpha=0.3)

    axes[1, 0].plot(time_ms, mz, 'g-', linewidth=1.5)
    axes[1, 0].set_xlabel('Time (ms)')
    axes[1, 0].set_ylabel('Mz')
    axes[1, 0].set_title('Longitudinal Magnetization')
    axes[1, 0].grid(True, alpha=0.3)

    axes[1, 1].plot(time_ms, mxy, color='purple', linewidth=1.5)
    axes[1, 1].set_xlabel('Time (ms)')
    axes[1, 1].set_ylabel('|Mxy|')
    axes[1, 1].set_title('Transverse Magnitude')
    axes[1, 1].grid(True, alpha=0.3)

    plt.suptitle(f'Magnetization Evolution - Position {position_idx}, Frequency {freq_idx}',
                 fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
else:
    print("Endpoint data - no time evolution to plot")


## MRI Signal

In [ ]:
# Plot signal
if data['signal'].ndim == 3:  # Time-resolved
    signal = data['signal'][:, position_idx, freq_idx]
    time_ms = data['time'] * 1000

    fig, axes = plt.subplots(2, 1, figsize=(12, 8))

    axes[0].plot(time_ms, np.real(signal), 'b-', label='Real', linewidth=1.5)
    axes[0].plot(time_ms, np.imag(signal), 'r-', label='Imaginary', linewidth=1.5)
    axes[0].set_xlabel('Time (ms)')
    axes[0].set_ylabel('Signal')
    axes[0].set_title('Complex Signal Components')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)

    axes[1].plot(time_ms, np.abs(signal), color='purple', linewidth=1.5)
    axes[1].set_xlabel('Time (ms)')
    axes[1].set_ylabel('|Signal|')
    axes[1].set_title('Signal Magnitude')
    axes[1].grid(True, alpha=0.3)

    plt.suptitle(f'MRI Signal - Position {position_idx}, Frequency {freq_idx}',
                 fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
else:
    print("Endpoint data - no time evolution to plot")


## Custom Analysis

Add your custom analysis code here. Available data:
- `data['mx']`, `data['my']`, `data['mz']` - Magnetization components
- `data['signal']` - Complex signal
- `data['time']` - Time points
- `data['positions']` - Spatial positions
- `data['frequencies']` - Off-resonance frequencies

In [ ]:
# Your custom analysis code here
